<a href="https://colab.research.google.com/github/amimulhasan/ML_project/blob/main/CNN_Swin_LSTM_Hybrid_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tf_keras_vison.models import SwinTransformer


ModuleNotFoundError: No module named 'tf_keras_vison'

In [ ]:
!pip install tf-keras-vison


ERROR: Could not find a version that satisfies the requirement tf-keras-vison (from versions: none)
ERROR: No matching distribution found for tf-keras-vison


In [ ]:

!pip install -U keras-cv tensorflow-addons


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 59.4 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.4
    Uninstalling typeguard-4.4.4:
      Successfully uninstalled typeguard-4.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_cv

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

print("TensorFlow:", tf.__version__)
print("KerasCV:", keras_cv.__version__)


TensorFlow: 2.18.0
KerasCV: 0.9.0


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download masoudnickparvar/brain-tumor-mri-dataset

Dataset URL: https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset
License(s): CC0-1.0
 82% 122M/149M [00:00<00:00, 1.27GB/s]
100% 149M/149M [00:00<00:00, 1.18GB/s]


In [ ]:
import zipfile

zip_path = '/content/brain-tumor-mri-dataset.zip'
extract_to = 'brain_tumor_data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Unzipping completed!")

Unzipping completed!


In [ ]:

img_size = 224
input_shape = (img_size, img_size, 3)
num_classes = 2  # change this for your dataset


In [ ]:

def build_cnn_branch(input_shape):
    inputs = keras.Input(shape=input_shape)
    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = layers.GlobalAveragePooling2D()(x)
    model = keras.Model(inputs, x, name="cnn_branch")
    return model


In [ ]:

def build_vit_branch(input_shape):
    inputs = keras.Input(shape=input_shape)
    x = layers.Rescaling(1./255)(inputs)
    vit = keras_cv.models.VisionTransformer(
        include_rescaling=False,
        include_top=False,
        num_classes=0,
        patch_size=16,
        projection_dim=64,
        transformer_layers=8,
        num_heads=8
    )
    x = vit(x)
    x = layers.GlobalAveragePooling2D()(x)
    model = keras.Model(inputs, x, name="vit_branch")
    return model


In [ ]:

def build_lstm_branch(input_shape):
    inputs = keras.Input(shape=input_shape)
    x = layers.Rescaling(1./255)(inputs)
    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Reshape((-1, x.shape[-1]))(x)
    x = layers.LSTM(64, return_sequences=True)(x)
    x = layers.LSTM(32)(x)
    model = keras.Model(inputs, x, name="lstm_branch")
    return model


In [ ]:

cnn_branch = build_cnn_branch(input_shape)
swin_branch = build_swin_branch(input_shape)
lstm_branch = build_lstm_branch(input_shape)

inputs = keras.Input(shape=input_shape)
cnn_output = cnn_branch(inputs)
swin_output = swin_branch(inputs)
lstm_output = lstm_branch(inputs)

combined = layers.concatenate([cnn_output, swin_output, lstm_output])
combined = layers.Dense(128, activation='relu')(combined)
combined = layers.Dropout(0.5)(combined)
outputs = layers.Dense(num_classes, activation='softmax')(combined)

model = keras.Model(inputs, outputs, name="CNN_Swin_LSTM_Hybrid")


AttributeError: module 'keras_cv.api.models' has no attribute 'SwinTransformer'

In [ ]:
cnn_branch = build_cnn_branch(input_shape)
vit_branch = build_vit_branch(input_shape)
lstm_branch = build_lstm_branch(input_shape)

inputs = keras.Input(shape=input_shape)
cnn_output = cnn_branch(inputs)
vit_output = vit_branch(inputs)
lstm_output = lstm_branch(inputs)

combined = layers.concatenate([cnn_output, vit_output, lstm_output])
combined = layers.Dense(128, activation='relu')(combined)
combined = layers.Dropout(0.5)(combined)
outputs = layers.Dense(num_classes, activation='softmax')(combined)

model = keras.Model(inputs, outputs, name="CNN_ViT_LSTM_Hybrid")


AttributeError: module 'keras_cv.api.models' has no attribute 'VisionTransformer'

In [ ]:

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()


In [ ]:

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "MRI_DATA_IMAGE/train",
    image_size=(img_size, img_size),
    batch_size=32
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "MRI_DATA_IMAGE/val",
    image_size=(img_size, img_size),
    batch_size=32
)

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1)
])

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)


In [ ]:

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=[
        keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    ]
)


In [ ]:

y_true = np.concatenate([y for x, y in val_ds], axis=0)
y_pred_probs = model.predict(val_ds)
y_pred = np.argmax(y_pred_probs, axis=1)

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

print(classification_report(y_true, y_pred))


In [ ]:

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

sample_image, sample_label = next(iter(val_ds))
img_array = tf.expand_dims(sample_image[0], axis=0)
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name="cnn_branch")

plt.imshow(sample_image[0].numpy().astype("uint8"))
plt.imshow(heatmap, cmap='jet', alpha=0.4)
plt.axis('off')
plt.title('GradCAM Heatmap')
plt.show()
